In [1]:
import pyvisa                                     # Required to talk to Keithley
import serial                                   # Required to connect to USB Devices
import msvcrt                                   # required to detect keyboard press
import time                                     # Required to sleep
import os                                       # Required to change directory
import datetime                                 # Required to figure out current system time
import math                                     # Required to do complex math
import csv                                      # Required to save to .csv files
import numpy as np                              # Required for arrays 
import pandas as pd                             # Required for Data Frames
from tkinter import *                           #
from tkinter.filedialog import askopenfilename  # Required for choosing file location
from tkinter.filedialog import askdirectory     #
from matplotlib import pyplot as plt            # Required for plots
import matplotlib.ticker as ticker              # Required for grid on plots
import datetime                                 # Required to log time of errors
#%matplotlib

In [2]:
print("\n")
print("#############################################################")
print("###                                                       ###")
print("###    Press ENTER to choose the folder for where the     ###")
print("###            lifetime_test current measurement          ###")
print("###                 data will be stored.                  ###")
print("###                                                       ###")
print("#############################################################")
file = input("")            # Waits for user to press enter before File 
foldername = askdirectory() # Opens File Explorer to choose and save file location
os.chdir(foldername)        # Changes the directory to chosen file location
Tk().destroy()             # Closes the main Tk wiindow



#############################################################
###                                                       ###
###    Press ENTER to choose the folder for where the     ###
###            lifetime_test current measurement          ###
###                 data will be stored.                  ###
###                                                       ###
#############################################################



In [3]:
rm=pyvisa.ResourceManager()
rm.list_resources()

('TCPIP0::129.107.83.133::inst0::INSTR',
 'ASRL1::INSTR',
 'ASRL2::INSTR',
 'ASRL4::INSTR',
 'ASRL6::INSTR',
 'ASRL7::INSTR',
 'ASRL8::INSTR')

In [4]:
my_instrument = rm.open_resource('ASRL7::INSTR',read_termination='\r')
time.sleep(5)
print(my_instrument.query("*IDN?"))

KEITHLEY INSTRUMENTS INC.,MODEL 6485,4125305,C01   Jun 23 2010 12:22:00/A02  /J


In [ ]:
time.sleep(5)
my_instrument.write("TRAC:CLE")
my_instrument.write("ARM:SOUR IMM")
my_instrument.write("ARM:COUN 1")
my_instrument.write("TRIG:SOUR IMM")
my_instrument.write("TRIG:COUN 1")
Sum = 0
k = 0
file_num = 0
Start = float(my_instrument.query("READ?").split(",")[1].replace("+", ""))
print("Press Enter in the command prompt to exit the data acquisition")
while (1):
    Time = []
    Current = []
    for i in range(0,100):
        my_instrument.write("TRAC:CLE")
        X = my_instrument.query("READ?")
        curr=float(X.split(",")[0].replace("+", "").replace("A",""))
        tim=float(X.split(",")[1].replace("+", ""))
        if(curr < 1.0*10**-11 or curr > 2.0*10**-11):
            Log = open("Error_Log.txt", "a")
            Log.write("{} {}\n".format(curr,datetime.datetime.now()))
            Log.close()
        Sum += curr
        if(i != 0 or k>0):
            if(k==0):
                Avg = Sum/i
            else:
                Avg = Sum/(i+(k-1))
        Current.append(curr)
        Time.append(tim - Start)
        time.sleep(1)
        if(i == 99):
            k = k+i+1
            plt.figure(figsize = (12,8))
            plt.xlim(Time[0],Time[len(Time)-1]+2.5)
            plt.plot(Time, Current)
            plt.savefig('Keithley_Time_Current_{}.png'.format(file_num))
            plt.close()
            Save = np.array(pd.concat([pd.DataFrame(Current), pd.DataFrame(Time)], axis=1))
            np.savetxt('current_time{}.txt'.format(file_num), Save)
            Avg_log = open("Current Avg Log.txt", "a")
            Avg_log.write("{} {}\n".format(Avg, datetime.datetime.now()))
            Avg_log.close()
    file_num = file_num + 1
    for i in range (0,100):
        enter_key=0
        #print(enter_key)
        if msvcrt.kbhit():
            if ord(msvcrt.getch()) == 13:
                enter_key = ord(msvcrt.getch())
                break
        time.sleep(3)
    if (enter_key == 13):
        break 

Press Enter in the command prompt to exit the data acquisition


In [ ]:
time.sleep(2)
rm.close()

In [ ]:
msvcrt.getch()